In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [2]:
df2 = df2.apply(pd.to_numeric, errors = 'coerce')
df2.describe()
# 없는 Column = codesum, store number, station_nbr, date

,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,...,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,9.512700e+04,95016.000000,95016.000000,93906.000000,92019.000000,94461.000000,93462.000000,93906.000000,93906.000000,93240.000000,...,94683.000000,95127.000000,95127.000000,95127.000000,95127.0,95127.0,95127.000000,95127.000000,95127.000000,95127.000000
mean,1.365321e+18,58.335280,41.850467,50.443262,2.474065,38.504113,44.890736,16.197400,1.640662,539.286905,...,9.567292,0.499417,0.499417,0.065344,16.0,2.0,56.000000,1.743469,3.004667,0.287048
std,2.520425e+16,18.426408,16.804727,17.431240,7.235081,18.914861,16.276571,15.422198,3.585312,101.674721,...,3.631573,0.500002,0.500002,0.247134,0.0,0.0,32.041808,15.870756,2.004085,0.452386
min,1.325376e+18,11.000000,-2.000000,6.000000,-18.000000,-12.000000,3.000000,0.000000,0.000000,411.000000,...,1.700000,0.000000,0.000000,0.000000,16.0,2.0,1.000000,0.000000,0.000000,0.000000
25%,1.343866e+18,44.000000,29.000000,36.000000,-2.000000,23.000000,32.000000,0.000000,0.000000,436.750000,...,6.900000,0.000000,0.000000,0.000000,16.0,2.0,28.000000,0.000000,1.000000,0.000000
50%,1.362442e+18,61.000000,43.000000,52.000000,2.000000,40.000000,47.000000,13.000000,0.000000,530.000000,...,9.200000,0.000000,0.000000,0.000000,16.0,2.0,56.000000,0.000000,3.000000,0.000000
75%,1.384474e+18,74.000000,56.000000,65.000000,7.000000,54.000000,58.000000,29.000000,0.000000,627.000000,...,11.700000,1.000000,1.000000,0.000000,16.0,2.0,84.000000,0.000000,5.000000,1.000000
max,1.414714e+18,93.000000,74.000000,84.000000,31.000000,72.000000,75.000000,59.000000,19.000000,716.000000,...,23.600000,1.000000,1.000000,1.000000,16.0,2.0,111.000000,577.000000,6.000000,1.000000


In [3]:
df2_drop_columns = ['date', 'store_nbr', 'station_nbr', 'codesum']
df2 = df2.drop(columns = df2_drop_columns)

In [4]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df2_columns = df2.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df2_columns:
    if (i == 'resultdir'):
        df2[i].fillna(df2[i].mode()[0], inplace=True)
        print(df2[i].mode()[0])
    else:
        df2[i].fillna(df2[i].mean(), inplace=True)

27.0


In [5]:
# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df2['relative_humility'] = 100*(np.exp((17.625*((df2['dewpoint']-32)/1.8))/(243.04+((df2['dewpoint']-32)/1.8)))/np.exp((17.625*((df2['tavg']-32)/1.8))/(243.04+((df2['tavg']-32)/1.8))))

# 체감온도 계산
df2["windchill"] = 35.74 + 0.6215*df2["tavg"] - 35.75*(df2["avgspeed"]**0.16) + 0.4275*df2["tavg"]*(df2["avgspeed"]**0.16)

In [6]:
df2 = df2[df2['units'] != 0]

In [7]:
df2.head()

,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,...,avgspeed,nothing,rain,other,item_nbr,units,week7,weekend,relative_humility,windchill
103134,48.0,33.0,41.0,16.0,37.0,39.0,24.0,0.0,716.0,1626.0,...,11.3,0,1,0,16,25,6,1,85.545628,34.361840
103143,48.0,33.0,41.0,16.0,37.0,39.0,24.0,0.0,716.0,1626.0,...,11.3,0,1,0,25,187,6,1,85.545628,34.361840
103245,46.0,28.0,37.0,12.0,24.0,32.0,28.0,0.0,716.0,1627.0,...,13.3,1,0,0,16,42,0,0,59.009292,28.579201
103254,46.0,28.0,37.0,12.0,24.0,32.0,28.0,0.0,716.0,1627.0,...,13.3,1,0,0,25,102,0,0,59.009292,28.579201
103268,46.0,28.0,37.0,12.0,24.0,32.0,28.0,0.0,716.0,1627.0,...,13.3,1,0,0,39,1,0,0,59.009292,28.579201


In [21]:
model_df2 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + wetbulb + heat + cool + preciptotal + stnpressure + \
sealevel + resultspeed + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df2)
result_df2 = model_df2.fit()

print(result_df2.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.927
Model:                            OLS   Adj. R-squared:                  0.926
Method:                 Least Squares   F-statistic:                     1050.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:00:41   Log-Likelihood:                -1457.1
No. Observations:                2515   AIC:                             2976.
Df Residuals:                    2484   BIC:                             3157.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(nothing)[0]        -0.0794      0.15

In [11]:
anova_result_df2 = sm.stats.anova_lm(result_df2, typ=2).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df2[anova_result_df2['PR(>F)'] <= 0.05]

,sum_sq,df,F,PR(>F)
cool,1.018454,1.0,5.392599,2.030279e-02
tmax,1.189672,1.0,6.299176,1.214254e-02
heat,1.251040,1.0,6.624115,1.011820e-02
C(nothing),1.802219,2.0,4.771272,8.547381e-03
tavg,2.033893,1.0,10.769234,1.046296e-03
C(weekend),7.343619,1.0,38.883630,5.269819e-10
C(week7),40.394499,6.0,35.647382,1.386762e-41
C(item_nbr),5899.740454,7.0,4462.636682,0.000000e+00


In [13]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df2.values, i) for i in range(df2.shape[1])]
vif["features"] = df2.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# item_nbr, weekend, week7, 

,VIF Factor,features
0,1.270759,units
1,1.279239,item_nbr
2,1.482039,snowfall
3,1.504612,resultdir
4,1.529649,other
5,1.542401,preciptotal
6,2.064454,depart
7,2.739358,weekend
8,2.746827,week7
9,10.343310,resultspeed


In [23]:
model_df2_1 = sm.OLS.from_formula('np.log1p(units) ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + \
scale(wetbulb) + scale(heat) + scale(cool) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed) + scale(resultdir) + \
scale(avgspeed) + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df2)
result_df2_1 = model_df2_1.fit()

print(result_df2_1.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.927
Model:                            OLS   Adj. R-squared:                  0.926
Method:                 Least Squares   F-statistic:                     1050.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:01:15   Log-Likelihood:                -1457.1
No. Observations:                2515   AIC:                             2976.
Df Residuals:                    2484   BIC:                             3157.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [24]:
# item_nbr, weekend, week7, 
model_df2_2 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + C(week7) + C(weekend) + 0', data = df2)
result_df2_2 = model_df2_2.fit()

print(result_df2_2.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.924
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     2348.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:03:01   Log-Likelihood:                -1501.3
No. Observations:                2515   AIC:                             3031.
Df Residuals:                    2501   BIC:                             3112.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
C(item_nbr)[16]     3.4054      0.026    1

In [32]:
from sklearn.model_selection import cross_val_score

X2 = df2[['week7', 'weekend', 'item_nbr']]
y2 = df2['units']
model2 = LinearRegression()

cv2 = KFold(n_splits=10, shuffle=True, random_state=0)

cross_val_score(model2, X2, y2, scoring="r2", cv=cv2)

array([0.21806679, 0.15769808, 0.20015997, 0.25939593, 0.17127481,
       0.2058334 , 0.22141082, 0.19673892, 0.19292167, 0.22319767])